In [1]:
import numpy as np
import pandas as pd
import itertools
import pickle
import sys
import os
import dsc
from dsc.query_engine import Query_Processor as dscQP
from dsc import dsc_io

import utils

In [29]:
#dsc_output = "/gpfs/commons/groups/knowles_lab/sbanerjee/sparse-regression/gradvi-experiments/linreg_indep"
dsc_output = "/gpfs/commons/groups/knowles_lab/sbanerjee/sparse-regression/gradvi-experiments/linreg_blockdiag"
#dsc_output = "/gpfs/commons/groups/knowles_lab/sbanerjee/sparse-regression/gradvi-experiments/trendfiltering"

dsc_fname  = os.path.basename(os.path.normpath(dsc_output))
db = os.path.join(dsc_output, dsc_fname + ".db")
respkl   = os.path.join("../dsc/results", dsc_fname + ".pkl")
dscoutpkl = os.path.join("../dsc/results", dsc_fname + "_dscout.pkl")

if os.path.isfile(dscoutpkl):
    dscout = pd.read_pickle(dscoutpkl)
else:
    print ("Could not find dscout from dscquery")

refresh_pickle = True
is_trendfiltering = False

if is_trendfiltering:
    target = ["simulate", "simulate.sfix", "simulate.snr", "simulate.n", "simulate.dtrue", "fit"]
else:
    target = ["simulate", "simulate.sfix", "simulate.pve",   "simulate.se", "simulate.dims", "fit"]

#condition = ["simulate.sfix == 2", "simulate.signal == 'normal'", "simulate.dims == '(50, 200)'"]
#groups = ["fit_cpt:"]
condition = [""]

print ("Reading from DSC database:")
print (f"    {db}")
print ("")

qp = dscQP(db, target, condition)
df = qp.output_table

df

Reading from DSC database:
    /gpfs/commons/groups/knowles_lab/sbanerjee/sparse-regression/gradvi-experiments/linreg_blockdiag/linreg_blockdiag.db



,DSC,simulate,simulate.sfix,simulate.se:output,simulate.pve,simulate.output.file,simulate.dims,fit,fit.output.file
0,1,blockdiag,5,blockdiag/blockdiag_5,0.6,blockdiag/blockdiag_5,"(500,10000)",mr_ash,mr_ash/blockdiag_5_mr_ash_1
1,1,blockdiag,10,blockdiag/blockdiag_6,0.6,blockdiag/blockdiag_6,"(500,10000)",mr_ash,mr_ash/blockdiag_6_mr_ash_1
2,1,blockdiag,20,blockdiag/blockdiag_7,0.6,blockdiag/blockdiag_7,"(500,10000)",mr_ash,mr_ash/blockdiag_7_mr_ash_1
3,1,blockdiag,250,blockdiag/blockdiag_8,0.6,blockdiag/blockdiag_8,"(500,10000)",mr_ash,mr_ash/blockdiag_8_mr_ash_1
4,2,blockdiag,5,blockdiag/blockdiag_17,0.6,blockdiag/blockdiag_17,"(500,10000)",mr_ash,mr_ash/blockdiag_17_mr_ash_1
...,...,...,...,...,...,...,...,...,...
475,19,blockdiag,250,blockdiag/blockdiag_156,0.6,blockdiag/blockdiag_156,"(500,10000)",gradvi_compound_lasso_init,gradvi_compound_lasso_init/blockdiag_156_lasso...
476,20,blockdiag,5,blockdiag/blockdiag_157,0.6,blockdiag/blockdiag_157,"(500,10000)",gradvi_compound_lasso_init,gradvi_compound_lasso_init/blockdiag_157_lasso...
477,20,blockdiag,10,blockdiag/blockdiag_158,0.6,blockdiag/blockdiag_158,"(500,10000)",gradvi_compound_lasso_init,gradvi_compound_lasso_init/blockdiag_158_lasso...
478,20,blockdiag,20,blockdiag/blockdiag_159,0.6,blockdiag/blockdiag_159,"(500,10000)",gradvi_compound_lasso_init,gradvi_compound_lasso_init/blockdiag_159_lasso...


In [30]:
dscout

,DSC,simulate,simulate.dims,simulate.se,simulate.sfix,simulate.pve,fit,fit.DSC_TIME,mse.err,coef_mse.err
0,1,blockdiag,"(500,10000)",1.641043,5,0.6,mr_ash,5.803000,3.125881,NaN
1,1,blockdiag,"(500,10000)",1.689415,10,0.6,mr_ash,6.295000,3.750698,NaN
2,1,blockdiag,"(500,10000)",3.099127,20,0.6,mr_ash,11.365000,11.993973,NaN
3,1,blockdiag,"(500,10000)",15.767455,250,0.6,mr_ash,12.307000,272.669176,NaN
4,2,blockdiag,"(500,10000)",0.963873,5,0.6,mr_ash,6.448000,1.195821,NaN
...,...,...,...,...,...,...,...,...,...,...
955,19,blockdiag,"(500,10000)",4.475719,250,0.6,gradvi_compound_lasso_init,54.264484,NaN,0.023303
956,20,blockdiag,"(500,10000)",1.725276,5,0.6,gradvi_compound_lasso_init,25.063624,NaN,0.000477
957,20,blockdiag,"(500,10000)",0.512876,10,0.6,gradvi_compound_lasso_init,58.607137,NaN,0.000124
958,20,blockdiag,"(500,10000)",2.079278,20,0.6,gradvi_compound_lasso_init,55.016513,NaN,0.001150


In [32]:
if is_trendfiltering:
    primary_keys     = ["simulate", "simulate.n", "simulate.sfix", "simulate.snr", "simulate.dtrue", "fit"]
    secondary_keys   = ["DSC"]
    target_stats     = ["tfmse", "niter", "elbo", "elboR", "runtime_wall", "runtime_cpu", "runtime_dsc", "convergence"]
    dataread_stats   = ["niter", "elbo", "elboR", "runtime_cpu", "runtime_wall", "convergence"]
    is_force_refresh = [False for x in target_stats]
else:
    primary_keys     = ["simulate", "simulate.dims", "simulate.sfix", "simulate.pve", "fit"]
    secondary_keys   = ["DSC"]
    target_stats     = ["mse", "coef_mse", "niter", "elbo", "runtime_wall", "runtime_cpu", "runtime_dsc", "convergence"]
    dataread_stats   = ["niter", "elbo", "runtime_cpu", "runtime_wall", "convergence"]
    is_force_refresh = [False for x in target_stats]

In [33]:
unique_rows = []

# First we create the unique value of the primary key
for pkey_uniq_vals in itertools.product(*[df[key].unique() for key in primary_keys]):    
    
    # Corresponding to each unique row of primary keys,
    # we find the unique secondary keys.
    p_rows = utils.stratify_dfcols(
        df, 
        [(key, val) for key, val in zip(primary_keys, pkey_uniq_vals)]
    )
    for skey_uniq_vals in itertools.product(*[p_rows[key].unique() for key in secondary_keys]):
        unique_rows.append(pkey_uniq_vals + skey_uniq_vals)

In [34]:
len(unique_rows)

480

In [35]:
resdict = {**{x: {} for x in primary_keys}, 
           **{x: {} for x in secondary_keys}, 
           **{x: {} for x in target_stats},
          }

resdict

{'simulate': {},
 'simulate.dims': {},
 'simulate.sfix': {},
 'simulate.pve': {},
 'fit': {},
 'DSC': {},
 'mse': {},
 'coef_mse': {},
 'niter': {},
 'elbo': {},
 'runtime_wall': {},
 'runtime_cpu': {},
 'runtime_dsc': {},
 'convergence': {}}

In [36]:
def read_statistic_from_dscout(colname, df, keys, vals):
    dfrows = utils.stratify_dfcols(df, [(k, v) for k, v in zip(keys, vals)])
    stat   = dfrows[~dfrows[colname].isnull()][colname].unique()
    if len(stat) == 1:
        return stat[0]
    else:
        print (f"Error fetching value for {colname}")
        print (vals)
        return stat[0]


def get_output_filename_from_db(outdir, row):
    method = row['fit'].values[0]
    fprefix = row['fit.output.file'].values[0]
    ext = 'pkl' if method.startswith('gradvi') else 'rds'
    fname = os.path.join(outdir, f"{fprefix}.{ext}")
    return fname


def read_output_file(outdir, db, keys, vals):
    dfrow = utils.stratify_dfcols(db, [(k, v) for k, v in zip(keys, vals)])
    '''
    must be a single row after selection
    '''
    assert(dfrow.shape[0] == 1)
    fname = get_output_filename_from_db(outdir, dfrow)
    data  = dsc_io.load_dsc(fname)
    return data


def get_niter_from_data(data):
    if 'fit' in data['model'].keys():
        if 'iter' in data['model']['fit'].keys():
            return data['model']['fit']['iter']
    if 'niter' in data['model'].keys():
        return data['model']['niter']
    
    
def get_runtime_from_data(data, rtype):
    rkey = f"runtime_{rtype}"
    if 'fit' in data['model'].keys():
        return data['model']['fit'][rkey]
    else:
        return data['model'][rkey]
    
    
def get_elbo_from_data(data, use_mrash = False):
    if 'fit' in data['model'].keys():
        return data['model']['fit']['varobj'][-1]
    else:
        if use_mrash:
            return data['model']['elbo']
        else:
            return data['model']['elbo_path'][-1]
    
    
def get_convergence_from_data(data, mrash_maxiter = 2000):
    '''
    1: converged
    2: abnormal termination
    3: number of iterations reached limit
    4: unknown
    '''
    if 'fit' in data['model'].keys():
        niter = data['model']['fit']['iter']
        if niter == mrash_maxiter:
            conv_status = 3
        else:
            conv_status = 1
            
    else:
        success = data['model']['success']
        if success:
            conv_status = 1
        else:
            conv_status = 4
            #conv_message = data['model']['fitobj']['message']
            #if "ITERATIONS REACHED LIMIT" in conv_message:
            #    conv_status = 3
            #elif "ABNORMAL_TERMINATION_IN_LNSRCH" in conv_message:
            #    conv_status = 2
            #else:
            #    conv_status = 4
        
    return conv_status

    
def read_statistic(stat, data, msedf, keys, vals):
    if stat == 'mse':        
        return read_statistic_from_dscout('mse.err', msedf, keys, vals)
    
    elif stat == 'coef_mse':
        return read_statistic_from_dscout('coef_mse.err', msedf, keys, vals)
    
    elif stat == 'tfmse':
        return read_statistic_from_dscout('tfmse.err', msedf, keys, vals)
    
    elif stat == 'niter':
        return get_niter_from_data(data)
    
    elif stat == 'runtime_wall':
        return get_runtime_from_data(data, 'wall')
    
    elif stat == 'runtime_cpu':
        return get_runtime_from_data(data, 'cpu')
    
    elif stat == 'elbo':
        return get_elbo_from_data(data)
    
    elif stat == 'elboR':
        return get_elbo_from_data(data, use_mrash = True)
    
    elif stat == 'convergence':
        return get_convergence_from_data(data)
    
    elif stat == 'runtime_dsc':
        return read_statistic_from_dscout('fit.DSC_TIME', msedf, keys, vals)

In [37]:
if refresh_pickle:
    
    is_data_read_required = [True if x in dataread_stats else False for x in target_stats]
    
    for i, row in enumerate(unique_rows):
        for j, key in enumerate(primary_keys + secondary_keys):
            resdict[key][i] = row[j]

    for i, row in enumerate(unique_rows):
        is_stat_missing = [False if i in resdict[stat].keys() else True for stat in target_stats]
        data_read_flags = [ x & (y | z) for x, y, z in zip(is_data_read_required, is_stat_missing, is_force_refresh)]

        data = None
        if any(data_read_flags):
            data = read_output_file(dsc_output, df, primary_keys + secondary_keys, row)

        for j, stat in enumerate(target_stats):
            if is_stat_missing[j] or is_force_refresh[j]:
                val = read_statistic(stat, data, dscout, primary_keys + secondary_keys, row)
                resdict[stat][i] = val

    resdf = pd.DataFrame.from_dict(resdict)
    resdf.to_pickle(respkl)
    
else:
    resdf = pd.read_pickle(respkl)

In [38]:
resdf

,simulate,simulate.dims,simulate.sfix,simulate.pve,fit,DSC,mse,coef_mse,niter,elbo,runtime_wall,runtime_cpu,runtime_dsc,convergence
0,blockdiag,"(500,10000)",5,0.6,mr_ash,1,3.125881,0.001191,82,996.923588,5.781000,5.635000,5.803000,1
1,blockdiag,"(500,10000)",5,0.6,mr_ash,2,1.195821,0.000508,83,789.008858,6.425000,6.180000,6.448000,1
2,blockdiag,"(500,10000)",5,0.6,mr_ash,3,0.194262,0.000090,87,353.585227,6.823000,6.621000,6.844000,1
3,blockdiag,"(500,10000)",5,0.6,mr_ash,4,9.630912,0.000679,137,1340.494262,9.208000,9.049000,9.229000,1
4,blockdiag,"(500,10000)",5,0.6,mr_ash,5,1.336688,0.000317,96,791.146657,7.234000,7.064000,7.255000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,blockdiag,"(500,10000)",250,0.6,gradvi_compound_lasso_init,16,106.134988,0.029508,122,1902.865289,46.958205,119.487641,46.958880,1
476,blockdiag,"(500,10000)",250,0.6,gradvi_compound_lasso_init,17,201.864307,0.028225,228,2073.917873,59.377506,178.951328,59.378308,1
477,blockdiag,"(500,10000)",250,0.6,gradvi_compound_lasso_init,18,215.852361,0.037187,154,2062.225463,52.713049,146.100938,52.713735,1
478,blockdiag,"(500,10000)",250,0.6,gradvi_compound_lasso_init,19,31.647101,0.023303,174,1599.092163,54.263902,144.750775,54.264484,1
